# 曲のチューニング確認と補正 (A=440Hzへ)
このノートブックでは、`librosa` を使用して楽曲の基準ピッチ (チューニング) がA=440Hzからどれくらいズレているかを測定し、自動的に補正する方法を解説します。

In [1]:
import librosa
import numpy as np
import soundfile as sf
import os
from pathlib import Path

# 作業ディレクトリの設定 (例: Demucsで分離されたフォルダの親、または元のデータ)
project_root = Path(os.getcwd()).parent
# data_dir = project_root / 'data' / 'processed' / 'htdemucs' / 'sample_kaze'
data_dir = project_root / 'data' / 'original' # 例としてオリジナル音源を使用

# 対象ファイルのパス
input_file = data_dir / 'sample_kaze.mp4'  # または 'vocals.mp3' など

print(f"Target file: {input_file}")

Target file: /Users/kpome/github/solfege-gen/data/original/sample_kaze.mp4


In [2]:
# 音声ファイルの読み込み
print("Loading audio...")
y, sr = librosa.load(input_file, sr=None)

# チューニングの推定
# estimate_tuning() は A=440Hz からのズレを「半音単位 (semitones)」で返します (±0.5 の範囲)
tuning_offset = librosa.estimate_tuning(y=y, sr=sr)

print(f"Estimated tuning offset: {tuning_offset:.4f} semitones")

# 周波数に換算してみる (A4 = 440Hz 基準)
estimated_freq = 440 * (2 ** (tuning_offset / 12))
print(f"Estimated Reference Frequency: {estimated_freq:.2f} Hz")

if abs(tuning_offset) < 0.01:
    print("The track is already well-tuned to A=440Hz.")
else:
    print(f"The track is detuned. Adjustment needed: {-tuning_offset:.4f} semitones.")

Loading audio...


/var/folders/rv/hmpprb6j6y38jxsvwy_sqp7c0000gn/T/ipykernel_33509/1999356622.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(input_file, sr=None)
/Users/kpome/github/solfege-gen/.venv/lib/python3.11/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Estimated tuning offset: -0.3300 semitones
Estimated Reference Frequency: 431.69 Hz
The track is detuned. Adjustment needed: 0.3300 semitones.


In [3]:
# ピッチ補正 (Shift)
# ズレを打ち消す方向にシフトします (n_steps = -tuning_offset)
if abs(tuning_offset) > 0.01:
    print("Applying pitch shift...")
    # librosa.effects.pitch_shift は高品質ですが処理が重い場合があります
    y_shifted = librosa.effects.pitch_shift(y, sr=sr, n_steps=-tuning_offset)
    
    # 補正後の確認
    new_offset = librosa.estimate_tuning(y=y_shifted, sr=sr)
    print(f"New tuning offset: {new_offset:.4f} semitones")
else:
    print("Skipping pitch shift.")
    y_shifted = y

Applying pitch shift...
New tuning offset: 0.0200 semitones


In [4]:
# ファイルの書き出し
# output_path = data_dir / 'sample_kaze_tuned_440.wav'  # librosa/soundfileはwav書き出し推奨

# sf.write(output_path, y_shifted, sr)
# print(f"Saved tuned audio to: {output_path}")

# Jupyter上で再生 (一部のみ)
from IPython.display import Audio
# 重いので最初の30秒だけプレビュー
Audio(data=y_shifted[:30*sr], rate=sr)